In [1]:
# !pip install flask langchain requests beautifulsoup4 faiss-cpu

In [1]:
from bs4 import BeautifulSoup
import requests

# Fetch the webpage content
url = "https://brainlox.com/courses/category/technical"
response = requests.get(url)
content = response.content

# Parse the content using BeautifulSoup
soup = BeautifulSoup(content, 'html.parser')

# Extract course data
def extract_course_data(soup):
    courses = []
    for course in soup.select('.single-courses-box'):
        title = course.select_one('.courses-content h3').text.strip() if course.select_one('.courses-content h3') else "No title"
        description = course.select_one('.courses-content p').text.strip() if course.select_one('.courses-content p') else "No description"
        price = course.select_one('.price-per-session-text').text.strip() if course.select_one('.price-per-session-text') else "No price"
        courses.append({'title': title, 'description': description, 'price': price})
    return courses

courses = extract_course_data(soup)

for course in courses:
    print(f"Title: {course['title']}")
    print(f"Description: {course['description']}")
    print(f"Price: {course['price']}")
    print("-----------")


Title: LEARN SCRATCH PROGRAMING
Description: Scratch Course is the foundation of coding and is a building block of a coding journey. If you want
Price: per session
-----------
Title: LEARN CLOUD COMPUTING BASICS-AWS
Description: In this course we are going to cover the basics and the most important services on AWS,
At the end
Price: per session
-----------
Title: LEARN MOBILE DEVELOPMENT
Description: Mobile application development is the process of creating software applications that run on a mobil
Price: per session
-----------
Title: LEARN CORE JAVA PROGRAMMING ONLINE
Description: Java is a very popular high-level, class-based, object-oriented programming language that is design
Price: per session
-----------
Title: LEARN ROBOTICS
Description: You can open all kinds of doors for advancement in so many careers with a basic understanding of el
Price: per session
-----------
Title: LEARN JAVASCRIPT
Description: JavaScript is the most popular programming language in the world. It powers 

In [3]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
import pickle

def create_embeddings(data):
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    texts = [item['description'] for item in data]
    vectors = embeddings.embed_documents(texts)  # Use the correct method
    
    # Create FAISS index and add vectors
    vector_store = FAISS.from_texts(texts, embeddings)
    
    # Save FAISS index and descriptions
    vector_store.save_local('faiss_index')
    with open('descriptions.pkl', 'wb') as f:
        pickle.dump(texts, f)

# Create embeddings and store them
create_embeddings(courses)



flask_code = """
from flask import Flask, request, jsonify
import pickle
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

app = Flask(__name__)

# Initialize the embedding model
model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Load the FAISS index and descriptions
vector_store = FAISS.load_local('faiss_index', embeddings=model, allow_dangerous_deserialization=True)
with open('descriptions.pkl', 'rb') as f:
    descriptions = pickle.load(f)

@app.route('/query', methods=['POST'])
def query():
    data = request.json
    user_query = data.get('query')
    
    # Generate embedding for the user query
    user_embedding = model.embed_documents([user_query])
    
    # Search the FAISS index
    results = vector_store.similarity_search(user_query, k=1)  # Retrieve top 1 result
    retrieved_descriptions = [result.page_content for result in results]
    
    return jsonify({"results": retrieved_descriptions})

if __name__ == '__main__':
    app.run(debug=False, port=5000)

"""

with open('app.py', 'w') as f:
    f.write(flask_code)

print("Flask app saved as app.py. You can run it separately using `python app.py`.")



Flask app saved as app.py. You can run it separately using `python app.py`.


In [30]:
import langchain.embeddings
print(dir(langchain.embeddings))


['Any', 'CacheBackedEmbeddings', 'DEPRECATED_LOOKUP', 'HypotheticalDocumentEmbedder', 'TYPE_CHECKING', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__getattr__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_import_attribute', 'cache', 'create_importer', 'logger', 'logging']


In [34]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings()
print(dir(embeddings))

['Config', '__abstractmethods__', '__annotations__', '__class__', '__class_vars__', '__config__', '__custom_root_type__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__exclude_fields__', '__fields__', '__fields_set__', '__format__', '__ge__', '__get_validators__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__include_fields__', '__init__', '__init_subclass__', '__iter__', '__json_encoder__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__post_root_validators__', '__pre_root_validators__', '__pretty__', '__private_attributes__', '__reduce__', '__reduce_ex__', '__repr__', '__repr_args__', '__repr_name__', '__repr_str__', '__rich_repr__', '__schema_cache__', '__setattr__', '__setstate__', '__signature__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__try_update_forward_refs__', '__validators__', '__weakref__', '_abc_impl', '_calculate_keys', '_copy_and_set_values', '_decompose_class', '_enforce_dict_if_root', '_get_value', '_init_pr

In [4]:
!pip install langchain faiss-cpu flask


In [2]:
!pip install flask